Text Generation with Markov Chains in Python
============================================

In [1]:
files = ['text/grimm_tales.txt']

text = ""
for f in files:
    with open(f, 'r') as f:
        text += f.read()
print(text[:500])

THE GOLDEN BIRD

A certain king had a beautiful garden, and in the garden stood a tree
which bore golden apples. These apples were always counted, and about
the time when they began to grow ripe it was found that every night one
of them was gone. The king became very angry at this, and ordered the
gardener to keep watch all night under the tree. The gardener set his
eldest son to watch; but about twelve o’clock he fell asleep, and in
the morning another of the apples was missing. Then the second


Weather as a Markov Chain
-------------------------

![alt text](images/markov_weather.png "Weather")

Matrix representation (rows are current state, columns are next state):

| | Sunny | Cloudy | Rainy |
| --- | --- | --- | --- |
| **Sunny** | 0.6 | 0.1 | 0.3 |
| **Cloudy** | 0.3 | 0.3 | 0.4 |
| **Rainy** | 0.3 | 0.2 | 0.5 |


Text as a Markov Chain
----------------------

**The cat ran over the dog.**

![alt text](images/markov_text1.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the | cat | ran | over | dog | . |
| --- | --- | --- | --- | --- | --- | --- |
| **the** | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| **cat** | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over** | 1 | 0 | 0 | 0 | 0 | 0 |
| **dog** | 0 | 0 | 0 | 0 | 0 | 1 |
| **.** | 0 | 0 | 0 | 0 | 0 | 1 |



Define states as the distinct word tokens

In [2]:
import re
text = re.sub("[^A-z,.!?'\n ]+", "", text)
text = re.sub("([.,!?])", r" \1 ", text)   #Resulting in much better tokenization
tokens = text.lower().split()
distinct_states = list(set(tokens)) #casting tokens to a set and then cast back to a list

Define transition matrix

In [3]:
# Here, we need to create data structures to hold our transistion matrix.
# So, if we were use to Numpy array, we could easily run out of memory.
# There can be many distinct words in a book, hence we need a very large transistion matrix.
# But thankfully, many of the entries will be zero as we saw in the sentence example above. 
# So, we can use a sparse array and this will store the non-zero entries in a very compact manner.

from scipy.sparse import csr_matrix #csr = compressed sparse row 
m = csr_matrix(
    (len(distinct_states), len(distinct_states)),
    dtype = int
)
state_index = dict(
    [(state, idx_num) for idx_num, state in enumerate(distinct_states)] #using python list comprehension
)

Count transitions and fill in transition matrix

In [4]:
for i in range(len(tokens)-1): #Since, we need not to worry about the last terminal state.
    row = state_index[tokens[i]] #Holding the current state
    col = state_index[tokens[i+1]] #State transitioning to next
    m[row,col] += 1

/home/rhyme/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Generate new text

In [5]:
import numpy as np

start_state_index = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]
num_sentences = 0
output = state.capitalize()
capitalize = False

while num_sentences < 3:
    row = m[state_index[state], :]
    probabilities = row / row.sum()
    probabilities = probabilities.toarray()[0]
    
    next_state_index = np.random.choice( #allows to sample according to probability distribution
        len(distinct_states),
        1,
        p = probabilities
    )
    next_state = distinct_states[next_state_index[0]]
    
    #Tokenizing our punctuation
    if next_state in ('.', '!', '?'):
        output += next_state + '\n\n'
        capitalize = True
        num_sentences += 1
    elif next_state == ",":
        output += next_state #ending o/p without a space
    else:
        if capitalize:
            output += next_state.capitalize()
            capitalize = False
        else:
            output += " " + next_state
            
    state = next_state
print(output)

Falada, and the wood, on with fatigue.

So afraid, my apples were fast asleep.

As he took her in the braids of the evil on the little maiden held her the poodle dog with no objection, we wish i did not that they were proud of the water, the queer, and he left side of the cat sitting in large dewdrop, thinking what these words that was now go again in my wife liked it fell to dance, all the other they were to himself on to the sparrow flew open it set his companions said hansel said the wind, she drove them will you shall suffer him, give me!




The format of the output is much better now, but the content is not great. One property of markov chain is that the next thing that happens only depends on the current state. So, that's why the grammar is so poor. Any two consecutive words makes sense together but then the next word over might not have any relation to that pair. So, one thing we can do to improve the result is to redefine the meaning of state to include the current token and the previous token. Here, down below, any 3 consecutive words will be related to each other and have the right grammar instead of just every pair of words and that could improve the grammar dramatically. We are going to call this a 2-Token/Word Markov Chain, since our states has 2 words in it.

k-Word Markov Chain
-------------------

**The cat ran over the dog.**

![alt text](images/markov_text2.png "Text")

Matrix representation (rows are current state, columns are next state):

| | the cat | cat ran | ran over | over the | the dog | dog. |
| --- | --- | --- | --- | --- | --- | --- |
| **the cat**  | 0 | 1 | 0 | 0 | 0 | 0 |
| **cat ran**  | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran over** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over the** | 0 | 0 | 0 | 0 | 1 | 0 |
| **the dog**  | 0 | 0 | 0 | 0 | 0 | 1 |
| **dog.**     | 0 | 0 | 0 | 0 | 0 | 1 |



Define states as consecutive token pairs

In [6]:
k = 2
tokens = text.lower().split()
#Here, we want to use states as keys in a dictonary but you can't use a list as a dictionary key. You must need something immutable. So, you can just cast that to a tuple.  
states = [tuple(tokens[i:i+k]) for i in range(len(tokens)-k+1)]
distinct_states = list(set(states))

Define and fill transition matrix

In [7]:
from scipy.sparse import csr_matrix
m = csr_matrix(
    (len(distinct_states), len(distinct_states)),
    dtype=int
)
state_index = dict(
    [(state, idx_num) for idx_num, state in enumerate(distinct_states)]
)

for i in range(len(tokens)-k):
    state = tuple(tokens[i:i+k])
    next_state = tuple(tokens[i+1:i+1+k])
    row = state_index[state]
    col = state_index[next_state]
    m[row,col] += 1

/home/rhyme/.local/lib/python3.6/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Generate new text

In [8]:
import numpy as np

start_state_index = np.random.randint(len(distinct_states))
state = distinct_states[start_state_index]
num_sentences = 0
output = ' '.join(state).capitalize() #Since state is a tuple of tokens, we need to join these into a string.
capitalize = False

while num_sentences < 3:
    row = m[state_index[state], :]
    probabilities = row / row.sum()
    probabilities = probabilities.toarray()[0]
    
    next_state_index = np.random.choice(
        len(distinct_states),
        1,
        p = probabilities
    )
    next_state = distinct_states[next_state_index[0]]
    
    if next_state[-1] in ('.', '!', '?'):
        output += next_state[-1] + '\n\n'
        capitalize = True
        num_sentences += 1
    elif next_state[-1] == ",":
        output += next_state[-1]
    else:
        if capitalize:
            output += next_state[-1].capitalize()
            capitalize = False
        else:
            output += " " + next_state[-1]
            
    state = next_state
print(output)

Be able to add considerably to their brother, and the cow and taking the pig was either bred or born but he was the matter.

There, and went her way.

So the brothers agreed that this golden slipper upon the ground, and yet i can have it so, to bring away such a youth has never harmed anyone?


